# <center> **CS 391, Spring 2021, Homework 7**
### <center> Due **Tuesday, March 23, 11:59 pm ET (Boston time)**, via Gradescope




###**Submission guidelines** 
Please write your solutions inside of this .ipynb file, then convert it to a PDF before submitting on Gradescope:

*   **In Jupyter:** File > Download as > PDF
*   **In Google Colab:** File > Print > Destination > Save as PDF

When you submit, please **be sure to match the answers on your PDF to the outline on Gradescope.** In other words, if the answer to problem 2.1 is on pages 2 and 3 of your PDF, please be sure to select those pages as the answer to problem 2.1 on Gradescope. Since it takes significantly longer to grade homework that is not properly matched, **we may deduct points** for noncompliant submissions.

The lab on Wednesday 1/27 covers how to get started with the notebooks for writing problem solutions and running experiments. In case you haven't done so, please sign up to the course Gradescope, with the access code **ERV7B2**: https://www.gradescope.com/courses/232562.

<div style="page-break-after: always;"></div>

**1. Data Analysis**

This week, we will again be using the Ames Housing data set, except this time using 20 numerical features to predict sales price:

In [14]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
from numpy.linalg import svd, norm
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

ames = pd.read_csv('https://raw.githubusercontent.com/catabia/cs391_spring21/main/AmesHousing.csv')
ames = ames[['Lot Frontage','Lot Area', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Gr Liv Area', 'Full Bath', 
             'Half Bath', 'Bedroom AbvGr', 'TotRms AbvGrd', 'BsmtFin SF 1', 'Bsmt Unf SF', 'Bsmt Full Bath',
             'Bsmt Half Bath', '1st Flr SF', '2nd Flr SF', 'Fireplaces', 'Garage Cars', 'Wood Deck SF',
             'Open Porch SF','SalePrice']]
ames.columns = ['lot_frontage','lot_area', 'overall_cond', 'year_built', 'year_remod', 'sqft', 'full_bath', 
                'half_bath', 'bedroom', 'total_rooms', 'bsmt_fin_sqft', 'bsmt_unfin_sqft', 'bsmt_full_bath',
                'bsmt_half_bath', 'first_fl_sqft', 'second_fl_sqft', 'fireplaces', 'garage_cars', 'wood_deck_sqft',
                'open_porch_sqft','sale_price']
ames=ames.dropna()
ames.describe()

,lot_frontage,lot_area,overall_cond,year_built,year_remod,sqft,full_bath,half_bath,bedroom,total_rooms,bsmt_fin_sqft,bsmt_unfin_sqft,bsmt_full_bath,bsmt_half_bath,first_fl_sqft,second_fl_sqft,fireplaces,garage_cars,wood_deck_sqft,open_porch_sqft,sale_price
count,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00,2437.00
mean,69.20,9696.03,5.57,1970.56,1984.45,1488.13,1.56,0.36,2.84,6.44,426.92,577.82,0.42,0.06,1152.37,330.65,0.56,1.75,88.99,47.04,180110.28
std,23.34,6383.02,1.10,31.54,21.37,504.12,0.55,0.50,0.82,1.58,463.28,444.69,0.52,0.24,393.82,421.55,0.63,0.78,120.69,67.94,83482.70
min,21.00,1300.00,1.00,1872.00,1950.00,334.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,334.00,0.00,0.00,0.00,0.00,0.00,12789.00
25%,58.00,7226.00,5.00,1950.00,1964.00,1124.00,1.00,0.00,2.00,5.00,0.00,229.00,0.00,0.00,866.00,0.00,0.00,1.00,0.00,0.00,127500.00
50%,68.00,9250.00,5.00,1972.00,1994.00,1433.00,2.00,0.00,3.00,6.00,338.00,490.00,0.00,0.00,1074.00,0.00,0.00,2.00,0.00,26.00,157500.00
75%,80.00,11207.00,6.00,2003.00,2004.00,1726.00,2.00,1.00,3.00,7.00,717.00,818.00,1.00,0.00,1381.00,689.00,1.00,2.00,168.00,68.00,212300.00
max,313.00,215245.00,9.00,2010.00,2010.00,5642.00,4.00,2.00,8.00,15.00,5644.00,2336.00,3.00,2.00,5095.00,2065.00,4.00,5.00,870.00,742.00,755000.00


Here is a simple method for figuring out how long it takes snippet of code to run in Python:

In [15]:
import time

start = time.time()
print('Hello word!')
end = time.time()
print('It took', end-start, ' seconds to print "Hello world!" once.  Pretty speedy, eh?')

Hello word!
It took 0.0001537799835205078  seconds to print "Hello world!" once.  Pretty speedy, eh?


**1.1** Normalize all features in the dataset to be between 0 and 1.  (This isn't necessary for OLS regression, but since we're predicting home sale prices, which tend to be very large, normalizing will keep our numbers small and easy to read).  Then, set up your $X$ and $y$ so that you may predict sale price using all other features in the dataset.

In [16]:
# Answer:
from sklearn.preprocessing import MinMaxScaler

#start by normalizing the data:
scaler = MinMaxScaler(feature_range=(0,1))

x = scaler.fit_transform(ames.drop(['sale_price'], axis=1))
y = scaler.fit_transform(ames.sale_price.values.reshape(ames.shape[0], 1))

**1.2**

In Lab 7, we learned how to analytically solve a linear regression using the normal equation.  The function LinearRegression from SciKit Learn (which we used on Homework 6) uses this technique.  Please compute the linear regression using this method, as well as the MSE of the model. How long does it take?

In [19]:
# Answer:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

start = time.time()
model = LinearRegression()
model.fit(x, y)
prediction = model.predict(x)
mse_a = mean_squared_error(prediction, y)
end = time.time()

print("Over all, the time it took us to compute the linear regression and MSE is ", end-start, " seconds.")
print("The MSE was: ", mse_a)

Over all, the time it took us to compute the linear regression and MSE is  0.014699697494506836  seconds.
The MSE was:  0.0027658972465704162


**1.3**

In Lab 8, we learned how to solve a linear regression using gradient descent.  The SciKit Learn function SGDRegressor also uses gradient descent.  Now, solve your linear regression using this method, with a learning rate of 0.0001 and a total of 100,000 iterations.  What is the MSE of this model?  How long does it take?  Is it significantly faster or slower than the normal equation method?

In [20]:
# Answer:
from sklearn.linear_model import SGDRegressor

start = time.time()
model = SGDRegressor(alpha = 0.0001, max_iter = 100000)
y=y.reshape(y.shape[0],)
model.fit(x, y)
mse_b = ((model.predict(X) - y)**2).mean()
end = time.time()
print("Over all, the time it took us to compute the MSE of the model is ", end-start, " seconds.")

print("The MSE was: ", mse_b)

Over all, the time it took us to compute the MSE of the model is  0.006687164306640625  seconds.
The MSE was:  0.004122761848742904


**Answer:**
We can see that using the linear regression method took us less time than the gradient-descent method (~40% of the time) to calculate the MSe, which is significantly less.

**1.4**

Of the two linear regression models you computed in **2.2** and **2.3**, which one has the lowest MSE?  Why?

**Answer:**

We notice that section **1.2** (the first time we computed the MSE) resulted in lower MSE (0027658972465704162) and **1.3** (where we used the gradient descent method) had a greater MSE (0.004122761848742904)

That's due to the number of iterations and learning rate perfromed when computing the MSE, Since the learning rate is fixed, and the number of the iterations could possibly be bigger (as it is probably in **1.2**), then our MSE is further away from 0 when using the gradient descent method - implying the model's greater imperfection in that case.

**1.5**

In Lab 7 and HW 6, we learned that we could solve a polynomial regression by fitting a linear regression to polynomial features (using SciKit learn's PolynomialFeatures method makes this simple).  Create cubic features (degree=3) for your $X$ data.  How many feature columns do you now have? Explain why there are so many.

In [32]:
# Answer:
from sklearn.preprocessing import PolynomialFeatures
model = PolynomialFeatures(3) 
x_degree3 = model.fit_transform(x)
model = LinearRegression()
model.fit(x_degree3, y)
prediction = model.predict(x_degree3)

print(len(x_degree3[0]))

print(len(x[0]))

1771
20


**Answer:**

We see that we have 1771 columns now in the 3rd degree polynomial regression of the provided dataset (for our X data).
Thet's due to the fact that we use a 3rd degree polynomial to approximate a 20 column matrix, or in other words, we take into account 20 categories and aspects of the dataset and use a regression (a 3rd degree polynomial regression) to approximate certain values based on that provided data - which has 20 (which is a lot) columns.

**1.6** Now, using both the normal equation (LinearResgression) and gradient descent (SGDRegressor) methods, compute a cubic regression of your data. Which method is faster?  Why?

In [34]:
# Answer:
degree = 3
# method 1 : Polynomial Regression
start1 = time.time()
model = PolynomialFeatures(degree) 
x_degree3 = model.fit_transform(x)
model = LinearRegression()
model.fit(x_degree3, y)
prediction = model.predict(x_degree3)
end1 = time.time()

# method 2 : Gradient Descent - CURRENTLY NOT SURE ABOUT
start2 = time.time()
model = SGDRegressor(alpha = 0.0001, max_iter = 100000)
y=y.reshape(y.shape[0],)
model.fit(x, y)
prediction = ((model.predict(X) - y)**degree)
end2 = time.time()


#summary:
print("Method 1 (Polynomial regression) took us : ", end1 - start1, " seconds.")
print("Method 2 (Gradient Descent) took us : ", end2 - start2, " seconds.")



Method 1 (Polynomial regression) took us :  2.869316816329956  seconds.
Method 2 (Gradient Descent) took us :  0.010333061218261719  seconds.


**Answer:**
For the third degree we see that the gradient descent method works significantly quicker.
Polynomial regression only works better than graident descent for a 2nd degree prediction (which is simply just a linear regression). That's simply due to the fact that creating such a large matrix for any degree larger than 2 takes much more time to traverse and calculate the necessary computation, where as in gradient descent the computation method is relied upon the use forumla, and therefore for any computation degree larger than 2 (3 or above), usng the gradient descent method is faster.